# TP1 - Problema 1: Horário Semanal para Reuniões de Projeto

Este notebook apresenta uma solução para o problema de construção de um horário semanal para reuniões de projeto de uma StartUp, utilizando a biblioteca pySMT para modelagem e resolução do problema.

In [ ]:
from pysmt.shortcuts import Symbol, And, Or, Not, Implies, Iff, ExactlyOne, Int, LE, GE, Plus, Equals, Int, get_model, Ite
from pysmt.typing import BOOL, INT
import itertools
from tabulate import tabulate

## Definição dos parâmetros do problema

In [ ]:
S = 2  # Número de salas
D = 5  # Número de dias
T = 4  # Número de slots de tempo por dia
P = 3  # Número de projetos
C = 10  # Número de colaboradores

# Dados fictícios para teste
projetos = {
    1: {'colaboradores': [1, 2, 3, 4], 'lider': 1, 'reunioes': 2},
    2: {'colaboradores': [3, 4, 5, 6, 7], 'lider': 3, 'reunioes': 3},
    3: {'colaboradores': [2, 5, 7, 8, 9, 10], 'lider': 2, 'reunioes': 2}
}

# Disponibilidade dos colaboradores (True se disponível, False se não)
disponibilidade = {c: [[True for _ in range(T)] for _ in range(D)] for c in range(1, C+1)}

# Dias da semana em português
dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]

## Criação das variáveis

Definimos duas variáveis de decisão:

1. $x_{p,s,d,t} \in \{0,1\}$: Indica se o projeto $p$ está agendado na sala $s$, no dia $d$, no slot $t$.
2. $y_{c,p,s,d,t} \in \{0,1\}$: Indica se o colaborador $c$ participa da reunião do projeto $p$ na sala $s$, no dia $d$, no slot $t$.

In [ ]:
x = {(p,s,d,t): Symbol(f'x_{p}_{s}_{d}_{t}', BOOL)
     for p in range(1, P+1)
     for s in range(1, S+1)
     for d in range(1, D+1)
     for t in range(1, T+1)}

y = {(c,p,s,d,t): Symbol(f'y_{c}_{p}_{s}_{d}_{t}', BOOL)
     for c in range(1, C+1)
     for p in range(1, P+1)
     for s in range(1, S+1)
     for d in range(1, D+1)
     for t in range(1, T+1)}

## Definição das restrições

1. Cada slot de tempo em cada sala pode ter no máximo uma reunião:
   $\forall s \in S, \forall d \in D, \forall t \in T: \sum_{p=1}^P x_{p,s,d,t} \leq 1$

2. Cada projeto deve ter o número correto de reuniões:
   $\forall p \in P: \sum_{s=1}^S \sum_{d=1}^D \sum_{t=1}^T x_{p,s,d,t} = R_p$
   onde $R_p$ é o número de reuniões requeridas para o projeto $p$.

3. Um colaborador só pode participar de uma reunião se estiver disponível:
   $\forall c \in C, \forall p \in P, \forall s \in S, \forall d \in D, \forall t \in T: \neg D_{c,d,t} \Rightarrow \neg y_{c,p,s,d,t}$
   onde $D_{c,d,t}$ é a disponibilidade do colaborador $c$ no dia $d$ e slot $t$.

4. Um colaborador só pode participar de uma reunião se for do projeto:
   $\forall c \in C, \forall p \in P, \forall s \in S, \forall d \in D, \forall t \in T: c \notin C_p \Rightarrow \neg y_{c,p,s,d,t}$
   onde $C_p$ é o conjunto de colaboradores do projeto $p$.

5. O líder do projeto deve participar de todas as reuniões do projeto:
   $\forall p \in P, \forall s \in S, \forall d \in D, \forall t \in T: x_{p,s,d,t} \Rightarrow y_{L_p,p,s,d,t}$
   onde $L_p$ é o líder do projeto $p$.

6. Pelo menos 50% dos colaboradores do projeto devem participar da reunião:
   $\forall p \in P, \forall s \in S, \forall d \in D, \forall t \in T: x_{p,s,d,t} \Rightarrow \sum_{c \in C_p} y_{c,p,s,d,t} \geq \lceil |C_p| / 2 \rceil$
   onde $|C_p|$ é o número de colaboradores do projeto $p$.

In [ ]:
constraints = []

# Restrição 1
for s in range(1, S+1):
    for d in range(1, D+1):
        for t in range(1, T+1):
            constraints.append(LE(Plus([Ite(x[p,s,d,t], Int(1), Int(0)) for p in range(1, P+1)]), Int(1)))

# Restrição 2
for p in projetos:
    constraints.append(Equals(Plus([Ite(x[p,s,d,t], Int(1), Int(0))
                                   for s in range(1, S+1)
                                   for d in range(1, D+1)
                                   for t in range(1, T+1)]),
                              Int(projetos[p]['reunioes'])))

# Restrição 3
for c in range(1, C+1):
    for p in range(1, P+1):
        for s in range(1, S+1):
            for d in range(1, D+1):
                for t in range(1, T+1):
                    if not disponibilidade[c][d-1][t-1]:
                        constraints.append(Not(y[c,p,s,d,t]))

# Restrição 4
for c in range(1, C+1):
    for p in projetos:
        if c not in projetos[p]['colaboradores']:
            for s in range(1, S+1):
                for d in range(1, D+1):
                    for t in range(1, T+1):
                        constraints.append(Not(y[c,p,s,d,t]))

# Restrição 5
for p in projetos:
    lider = projetos[p]['lider']
    for s in range(1, S+1):
        for d in range(1, D+1):
            for t in range(1, T+1):
                constraints.append(Implies(x[p,s,d,t], y[lider,p,s,d,t]))

# Restrição 6
for p in projetos:
    num_colaboradores = len(projetos[p]['colaboradores'])
    min_participantes = num_colaboradores // 2 + (num_colaboradores % 2)
    for s in range(1, S+1):
        for d in range(1, D+1):
            for t in range(1, T+1):
                constraints.append(Implies(x[p,s,d,t],
                                          GE(Plus([Ite(y[c,p,s,d,t], Int(1), Int(0))
                                                   for c in projetos[p]['colaboradores']]),
                                             Int(min_participantes))))

## Resolução do problema

In [ ]:
formula = And(constraints)
model = get_model(formula)

if model:
    print("Solução encontrada:")
    
    # Criar cabeçalho da tabela
    headers = ["Slots"] + dias_semana
    table_data = []
    
    for t in range(1, T+1):
        row = [f"Slot {t}"]
        for d in range(1, D+1):
            cell_content = ""
            for s in range(1, S+1):
                for p in range(1, P+1):
                    if model.get_py_value(x[p,s,d,t]):
                        participantes = [c for c in range(1, C+1) if model.get_py_value(y[c,p,s,d,t])]
                        cell_content += f"Sala {s}, P{p}: {', '.join(map(str, participantes))}\n"
            row.append(cell_content.strip())
        table_data.append(row)
    
    # Imprimir a tabela
    print(tabulate(table_data, headers=headers, tablefmt="grid"))
    print()
    
    # Legenda
    print("Legenda:")
    for p in range(1, P+1):
        print(f"P{p}: Projeto {p}")
        print(f"   Líder: {projetos[p]['lider']}")
        print(f"   Colaboradores: {', '.join(map(str, projetos[p]['colaboradores']))}")
        print()
else:
    print("Não foi possível encontrar uma solução.")